In [1]:
import yaml
import sqlite3
import pandas as pd
import scoped_mapping
import re
import requests
import string

import sys

import numpy as np
from sklearn.cluster import AffinityPropagation
import distance

mixs_terms_file = "../../mixs-source/model/schema/terms.yaml"
harmonized_table_db = "../target/harmonized_table.db"

# this is static, so counts will not be accuarate as adatabase grows
# raw env_package values could be added or removed over time, too
# I have code that automates some of this, but even that has SOME manual curations
# and some intermittent bugs (esp removing leading checklist name)
env_package_count_file = "../data/env_package_count_manual_20210824.tsv"

# there are some runons like
# 81 occurences of ...lhouseENVO:00003040
# start with a capital letter
obo_pattern = "[A-Z][a-zA-Z]{1,9}[\.| |_|:][0-9]{6,9}"

ols_ontologies_url = "http://www.ebi.ac.uk/ols/api/ontologies?size=500"
# only up to mid 200s now. Not bothering with pagination

findByIdAndIsDefiningOntology_url = (
    "http://www.ebi.ac.uk/ols/api/terms/findByIdAndIsDefiningOntology?obo_id="
)

output_file = "onto_slots_by_env_pack.tsv"

In [2]:
# works bare but not as a function?
# dumps objects in memory sorted by size

# https://stackoverflow.com/questions/40993626/list-memory-usage-in-ipython-and-jupyter
# These are the usual ipython objects, including this one you are creating
ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]

# Get a sorted list of the objects and their sizes
def var_sizes():
    temp = sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
    print(temp)
    return temp

In [3]:
with open(mixs_terms_file) as f:
    mixs_terms = yaml.safe_load(f)

mixs_slots = mixs_terms["slots"]

slotnames = mixs_slots.keys()

termid_terms = []

In [4]:
for slot in slotnames:
    current_obj = mixs_slots[slot]
    current_keys = list(current_obj.keys())
    if "pattern" in current_keys:
        the_pattern = current_obj["pattern"]
        if "termID" in the_pattern:
            termid_terms.append(slot)

In [5]:
termid_terms.sort()
termid_terms

['chem_administration',
 'env_broad_scale',
 'env_local_scale',
 'env_medium',
 'experimental_factor',
 'growth_facil',
 'host_body_product',
 'host_body_site',
 'host_disease_stat',
 'host_phenotype',
 'plant_growth_med',
 'plant_struc',
 'ploidy',
 'samp_collec_device',
 'season',
 'seq_meth']

In [6]:
connection = sqlite3.connect(harmonized_table_db)
cursor = connection.execute("select * from biosample limit 1")
ht_names = [description[0] for description in cursor.description]

In [7]:
htms = set(ht_names)
tts = set(termid_terms)
missing_from_ht = list(tts - htms)
missing_from_ht.sort()
missing_from_ht

['growth_facil',
 'host_body_site',
 'host_disease_stat',
 'plant_growth_med',
 'plant_struc',
 'samp_collec_device',
 'season',
 'seq_meth']

In [8]:
terms_to_tabulate = list(tts.intersection(htms))

### possible remedies?
- host_disease_stat (host_disease? host_disease_outcome?)
- plant_growth_med (there is a general growth_med )
- plant_struc (there is a plant_body_site)
- samp_collec_device spelled as samp_collect_device

### CJM doesn't care about these
- season (there is a season_environment)
- seq_meth

In [9]:
manual_adds = [
    "host_disease",
    "host_disease_outcome",
    "growth_med",
    "plant_body_site",
    "samp_collect_device",
]
terms_to_tabulate = terms_to_tabulate + manual_adds
terms_to_tabulate.sort()
terms_to_tabulate

['chem_administration',
 'env_broad_scale',
 'env_local_scale',
 'env_medium',
 'experimental_factor',
 'growth_med',
 'host_body_product',
 'host_disease',
 'host_disease_outcome',
 'host_phenotype',
 'plant_body_site',
 'ploidy',
 'samp_collect_device']

In [10]:
env_package_count_manual = pd.read_csv(env_package_count_file, sep="\t")

In [11]:
env_package_count_manual

,env_package,count,tidy,confidence,notes
0,air,234,air,high,NaN
1,MIGS/MIMS/MIMARKS.air,205,air,high,NaN
2,built environment,14510,built,high,NaN
3,built,61,built,high,NaN
4,host-associated,69981,host-associated,high,NaN
...,...,...,...,...,...
91,env_package,2,NaN,NaN,NaN
92,ENVO:00000016,1,NaN,NaN,NaN
93,MIMARKS_16S,1,NaN,NaN,NaN
94,saliva,1,NaN,NaN,NaN


In [12]:
# add env_package_count_manual to sqlite database

## indices!
# create index biosample_env_pack_idx on
#  biosample(env_package) ;

In [13]:
# build query from sample id, raw and tidy env_package, and the terms_to_tabulate columns
# then cast wide to long

for_query = ", ".join(terms_to_tabulate)

In [14]:
whole_query = (
    """select
	id,
	b.env_package as env_package_raw,
	epcm.tidy as env_package_tidy, """
    + for_query
    + """ from
	biosample b
join env_package_count_manual epcm on
	b.env_package = epcm .env_package
where
	epcm.tidy != ''
-- limit 9
"""
)

In [15]:
termterms_by_tidypackage = pd.read_sql_query(whole_query, connection)

In [16]:
wtl = pd.melt(
    termterms_by_tidypackage,
    id_vars="id",
    value_vars=terms_to_tabulate,
)

In [17]:
wtl = wtl[wtl["value"].notna()]

In [18]:
wtl["value"] = wtl["value"].str.split("|")
wtl = wtl.explode("value", ignore_index=True)

In [19]:
wtl_value_counts = wtl["value"].value_counts().to_frame()
wtl_value_counts.reset_index(level=0, inplace=True)
wtl_value_counts.columns = ["value", "count"]

In [20]:
for_capture = "(" + obo_pattern + ")"
p = re.compile(for_capture)
wtl_value_counts["extract"] = wtl_value_counts["value"].str.extract(p)
# are there any extracts that generate a list?
# now how to tidy and or validate? OLS lookup?

In [21]:
extract_count = wtl_value_counts["extract"].value_counts().to_frame()
extract_count.reset_index(level=0, inplace=True)
extract_count.columns = ["extract", "count"]

In [22]:
split_extract = extract_count["extract"].str.split(r"[ .:_]", expand=True)
split_extract.columns = ["prefix", "local_part"]

In [23]:
extract_count = pd.concat([extract_count, split_extract], axis=1)
extract_count["prefix_lower"] = extract_count["prefix"].str.lower()
extract_count["prefix_upper"] = extract_count["prefix"].str.upper()

# what if prefix doesn't ahve the right capitalizastion
# could lowercase, then look up as OLS ontologyId and get preferredPrefix

extract_count["reconstituted"] = extract_count[["prefix_upper", "local_part"]].agg(
    ":".join, axis=1
)

In [24]:
prefix_count = extract_count["prefix_lower"].value_counts().to_frame()
prefix_count.reset_index(level=0, inplace=True)
prefix_count.columns = ["prefix", "count"]

In [25]:
r = requests.get(ols_ontologies_url)
rj = json.loads(r.content)
# only up to mid 200s now. Not bothering with pagination
rj["page"]["totalPages"]

1

In [26]:
# ols_ontology_ids = []
ols_ontology_ids = {}
ols_ontology_list = rj["_embedded"]["ontologies"]
# dict comprehension?
for ontology in ols_ontology_list:
    #     ols_ontology_ids.append(ontology["ontologyId"])
    if "config" in ontology:
        if "preferredPrefix" in ontology["config"]:
            ols_ontology_ids[ontology["ontologyId"]] = ontology["config"][
                "preferredPrefix"
            ]
    else:
        ols_ontology_ids[ontology["ontologyId"]] = ""

## preferredPrefix says "NCBITAXON" not "NCBITaxon"

In [27]:
ols_ontology_id_list = list(ols_ontology_ids.keys())

In [28]:
claimed_prefixes = set(prefix_count["prefix"])

In [29]:
ols_doesnt_recognize = list(claimed_prefixes - set(ols_ontology_id_list))
ols_doesnt_recognize

['seawater', 'env']

In [30]:
claimed_recognized = list(claimed_prefixes.intersection(set(ols_ontology_id_list)))
claimed_recognized

['efo', 'pco', 'envo', 'eol', 'chebi', 'obi', 'foodon', 'uberon', 'po']

_could manually assert  env -> envo_

In [31]:
prefix_count["recognized"] = prefix_count["prefix"].isin(claimed_recognized)

In [32]:
temp = prefix_count[["prefix", "recognized"]]
temp.columns = ["prefix_lower", "recognized"]

In [33]:
extract_count = pd.merge(
    extract_count, temp, left_on="prefix_lower", right_on="prefix_lower", how="left"
)

In [34]:
extracted_recognized_list = extract_count["reconstituted"].loc[
    extract_count["recognized"]
]
extracted_recognized_list.sort_values(inplace=True)

Look up labels for term ids that could be parsed out of the annotations

In [35]:
# term_to_label = {}
# for term in extracted_recognized_list:
#     print(term)
#     r = requests.get(findByIdAndIsDefiningOntology_url + term)
#     if r.status_code == 200:
#         temp = json.loads(r.content)
#         if "_embedded" in temp:
#             if "terms" in temp["_embedded"]:
#                 # list. assume 0 or 1?
#                 #   as opposed to multiple hits for a single term in the defining ontology
#                 if "label" in temp["_embedded"]["terms"][0]:
#                     print(temp["_embedded"]["terms"][0]["label"])
#                     term_to_label[term] = temp["_embedded"]["terms"][0]["label"]

In [36]:
# use this as a cache to minimize OLS API REST calls
# term_to_label = pd.DataFrame(term_to_label.items(), columns=["id", "label"])

In [37]:
# term_to_label.to_sql(
#     "term_to_label",
#     connection,
#     if_exists="append",
#     index=False,
#     index_label=None,
#     chunksize=None,
#     dtype=None,
#     method=None,
# )

In [38]:
term_to_label = pd.read_sql(
    "select * from term_to_label",
    connection,
    index_col=None,
    coerce_float=False,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None,
)

In [39]:
extract_count = pd.merge(
    extract_count, term_to_label, left_on="reconstituted", right_on="id", how="left"
)

In [41]:
wtl_value_counts = pd.merge(
    wtl_value_counts,
    extract_count,
    how="left",
    on="extract",
    suffixes=("_annot", "_extr"),
)

In [42]:
wtl_value_counts["depleted"] = wtl_value_counts["value"].replace(
    value="", inplace=False, limit=None, regex=obo_pattern
)

In [43]:
term_to_label["tidy"] = term_to_label["label"]
term_to_label["tidy"] = term_to_label["tidy"].str.lower()

# replace punctuation
term_to_label["tidy"] = term_to_label["tidy"].str.replace(
    "[{}]".format(string.punctuation), " "
)

term_to_label["tidy"] = term_to_label["tidy"].str.strip()
term_to_label["tidy"] = term_to_label["tidy"].replace(
    value=" ", inplace=False, limit=None, regex=" +"
)

# refactor!

/var/folders/vt/f297tpjn6n1b6dwkcwhhnmxm0000gp/T/ipykernel_41992/1205103326.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  term_to_label["tidy"] = term_to_label["tidy"].str.replace(


In [44]:
# wtl_value_counts['tidy'] = scoped_mapping.whiteout...

wtl_value_counts["tidy"] = wtl_value_counts["depleted"]
wtl_value_counts["tidy"] = wtl_value_counts["tidy"].str.lower()

# replace punctuation
wtl_value_counts["tidy"] = wtl_value_counts["tidy"].str.replace(
    "[{}]".format(string.punctuation), " "
)

wtl_value_counts["tidy"] = wtl_value_counts["tidy"].str.strip()
wtl_value_counts["tidy"] = wtl_value_counts["tidy"].replace(
    value=" ", inplace=False, limit=None, regex=" +"
)

# replace or utilize ontology name prefixes (before text, not numbers)
preflist = list(prefix_count["prefix"].loc[prefix_count["recognized"]])
for prefix in preflist:
    anchored = "^" + prefix + " "
    wtl_value_counts["tidy"] = wtl_value_counts["tidy"].replace(
        value="", inplace=False, limit=None, regex=anchored
    )

# propigate labels that were already looked up from terms?
temp = term_to_label[["tidy", "id"]]
wtl_value_counts = pd.merge(
    wtl_value_counts,
    temp,
    left_on="tidy",
    right_on="tidy",
    how="left",
    suffixes=("_orig", "_backtrack"),
)

wtl_value_counts["id_consensus"] = wtl_value_counts["id_orig"].fillna(
    wtl_value_counts["id_backtrack"]
)

# don't pursue tidy terms that only appear once
#   but what about numerical suffixes like fish lung 1, fish lung 2, etc>

/var/folders/vt/f297tpjn6n1b6dwkcwhhnmxm0000gp/T/ipykernel_41992/2178637703.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  wtl_value_counts["tidy"] = wtl_value_counts["tidy"].str.replace(


In [45]:
wtl_value_counts.to_clipboard(index=False)

In [46]:
singletons = wtl_value_counts["tidy"].loc[
    wtl_value_counts["count_annot"] == 1 & wtl_value_counts["id_consensus"].isna()
]
singletons = singletons.value_counts().to_frame()
singletons.reset_index(level=0, inplace=True)
singletons.columns = ["value", "count"]

singletons["strlen"] = singletons["value"].str.len()
singletons["pursue"] = singletons["count"] == 1 & singletons["strlen"].gt(0)
singletons = singletons.loc[singletons["pursue"]]

singletons["generalized"] = singletons["value"].replace(
    value="", inplace=False, limit=None, regex="[ 0-9]+$"
)

generalized = singletons["generalized"].value_counts().to_frame()
generalized.reset_index(level=0, inplace=True)
generalized.columns = ["generalized", "count"]
generalized["strlen"] = generalized["generalized"].str.len()

singletons = pd.merge(
    singletons,
    generalized,
    left_on="generalized",
    right_on="generalized",
    how="left",
    suffixes=("_s", "_g"),
)

gen_useful = singletons.loc[singletons["count_g"].gt(1) & singletons["strlen_g"].gt(2)]

gen_useful = gen_useful[["value", "generalized"]]

# also remove " from.*$" ?

# words = np.asarray(temp['value'].loc[temp['use_for_clust']])

In [47]:
gen_useful.to_clipboard(index=False)

In [48]:
wtl_value_counts = pd.merge(
    wtl_value_counts, gen_useful, how="left", left_on="value", right_on="value"
)

In [49]:
wtl_value_counts["generalized_consensus"] = wtl_value_counts["generalized"].fillna(
    wtl_value_counts["tidy"]
)

In [50]:
wtl_value_counts.to_clipboard(index=False)

In [51]:
min_count = 10
elected_ontologies = "envo,micro,fma,uberon,ncbitaxon,foodon,ma,efo,chebi,agro,pato,doid,mondo,obi,pr,hp,po"

In [52]:
raw_list = (
    wtl_value_counts.loc[wtl_value_counts["id_consensus"].isna()]
    .groupby(by=["generalized_consensus"])
    .sum()
)

raw_list.reset_index(level=0, inplace=True)

raw_list = list(
    raw_list["generalized_consensus"].loc[raw_list["count_annot"].ge(min_count)]
)

list_len = len(raw_list)

# skip blanks or all numbers

In [53]:
# raw_list

This times out or errors out sometimes. Make sure computer doesn't go to sleep.

Cache it!

In [54]:
print(elected_ontologies)
print(min_count)
print(list_len)

sgaw_res = scoped_mapping.search_get_annotations_wrapper(
    raw_list,
    #     bad_chars=standard_replacement_chars,
    #     cat_name=standard_cat_name,
    ontoprefix=elected_ontologies,
    #     query_fields="",
    rr=10,
    string_dist_arg=2,
)

# ge 100 (329 queries) 7 min
#  was using gt, not gte
#  8 minutes for count > 30
# 12 minutes for count > 10 (1066 queries)
# XX minutes for count >  3 (1538 queries)
#    error

# not currently prioritizing by ontology
# so root may go to NCBItaxon

envo,micro,fma,uberon,ncbitaxon,foodon,ma,efo,chebi,agro,pato,doid,mondo,obi,pr,hp,po
10
1119


ConnectionError: ('Connection aborted.', OSError(50, 'Network is down'))

In [ ]:
# query	name	string_dist_rank	string_dist
# non saline sediment environment	saline sediment environment	1	0.045
# microbial mats	microbial mat	1	0.039

ba = scoped_mapping.get_best_acceptable(sgaw_res, max_string_dist=0.04)

print(min_count)
print(list_len)
print(len(ba.index))

In [ ]:
ols_mergable = ba[["raw", "obo_id", "label"]]
wtl_value_counts = pd.merge(
    wtl_value_counts,
    ols_mergable,
    left_on="generalized_consensus",
    right_on="raw",
    how="left",
    suffixes=("_presearch", "_withsearch"),
)

In [ ]:
wtl_value_counts["id_final"] = wtl_value_counts["obo_id"].fillna(
    wtl_value_counts["id_consensus"]
)

In [ ]:
wtl_value_counts["label_final"] = wtl_value_counts["label_withsearch"].fillna(
    wtl_value_counts["label_presearch"]
)

In [ ]:
wtl_value_counts["label_final"] = wtl_value_counts["label_final"].fillna(
    wtl_value_counts["generalized_consensus"]
)

In [ ]:
merge_back_to_samples = wtl_value_counts[["value", "id_final", "label_final"]]
merge_back_to_samples = merge_back_to_samples.loc[
    ~merge_back_to_samples["id_final"].isna()
]

In [ ]:
wtl = pd.merge(
    wtl,
    merge_back_to_samples,
    left_on="value",
    right_on="value",
    how="left",
    suffixes=("_sqlres", "_mappingres"),
)

In [ ]:
wtl["id_final"] = wtl["id_final"].fillna("unmapped")
wtl["label_final"] = wtl["label_final"].fillna("unmapped")

wtl["lab_id"] = wtl["label_final"] + " [" + wtl["id_final"] + "]"

In [ ]:
for_cast = wtl[["id", "variable", "lab_id"]]

In [ ]:
for_cast["lab_id"] = (
    for_cast[["id", "lab_id", "variable"]]
    .groupby(["id", "variable"])["lab_id"]
    .transform(lambda x: " | ".join(x))
)
for_cast = for_cast[["id", "variable", "lab_id"]].drop_duplicates()

In [ ]:
casted = for_cast.pivot(index="id", columns="variable", values="lab_id")

In [ ]:
casted.reset_index(level=0, inplace=True)

In [ ]:
temp = termterms_by_tidypackage[["id", "env_package_tidy"]]
temp.columns = ["id", "env_package"]

In [ ]:
casted = casted.drop("env_package_tidy", axis=1)

In [ ]:
casted = pd.merge(
    casted,
    temp,
    how="left",
    on="id",
    suffixes=("_slots", "_package"),
)

In [ ]:
put_first = ["id", "env_package"]
casted_cols = set(casted.columns)
casted_cols = list(casted_cols - set(put_first))
casted_cols.sort()
casted_cols = put_first + casted_cols

In [ ]:
casted = casted[casted_cols]

In [ ]:
casted.to_csv(output_file, index=False, sep="\t")

In [64]:
# casted = pd.read_csv("onto_slots_by_env_pack.tsv", sep="\t", low_memory=False)

put_first = ["env_package"]
omit_only = ["id"]
special_request = put_first + omit_only

for_melt = casted.columns

for_melt = list(set(for_melt) - set(special_request))
for_melt.sort()

casted_cols = put_first + for_melt
casted_cols

ignore_samples = casted[casted_cols]

,env_package,chem_administration,env_broad_scale,env_local_scale,env_medium,experimental_factor,growth_med,host_body_product,host_disease,host_phenotype,plant_body_site,ploidy,samp_collect_device
0,water,NaN,unmapped [unmapped],unmapped [unmapped],sea water [ENVO:00002149],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,water,NaN,unmapped [unmapped],unmapped [unmapped],sea water [ENVO:00002149],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,water,NaN,unmapped [unmapped],unmapped [unmapped],sea water [ENVO:00002149],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,water,NaN,unmapped [unmapped],unmapped [unmapped],sea water [ENVO:00002149],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,water,NaN,unmapped [unmapped],unmapped [unmapped],sea water [ENVO:00002149],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245058,human-associated,NaN,urban biome [ENVO:01000249],unmapped [unmapped],bodily fluid [UBERON:0006314],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245059,human-associated,NaN,urban biome [ENVO:01000249],unmapped [unmapped],bodily fluid [UBERON:0006314],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245060,human-associated,NaN,urban biome [ENVO:01000249],unmapped [unmapped],bodily fluid [UBERON:0006314],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245061,human-associated,NaN,urban biome [ENVO:01000249],unmapped [unmapped],bodily fluid [UBERON:0006314],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
is_melt = pd.melt(
    ignore_samples,
    id_vars="env_package",
    value_vars=for_melt,
)

is_melt = is_melt[is_melt["value"].notna()]

is_melt["placeholder"] = 1

,env_package,variable,value,placeholder
529,human-gut,chem_administration,unmapped [unmapped],1
530,human-gut,chem_administration,unmapped [unmapped],1
531,human-gut,chem_administration,unmapped [unmapped],1
532,human-gut,chem_administration,unmapped [unmapped],1
533,human-gut,chem_administration,unmapped [unmapped],1
...,...,...,...,...
2940703,human-gut,samp_collect_device,unmapped [unmapped],1
2940704,human-gut,samp_collect_device,unmapped [unmapped],1
2940705,human-gut,samp_collect_device,unmapped [unmapped],1
2940706,human-gut,samp_collect_device,unmapped [unmapped],1


In [74]:
is_counts = (
    is_melt[["env_package", "variable", "value", "placeholder"]]
    .groupby(["env_package", "variable", "value"])
    .agg(["count"])
)

is_counts.reset_index(inplace=True)
is_counts.columns = ["env_package", "variable", "value", "val_count"]

,env_package,variable,value,val_count
0,air,env_broad_scale,air [ENVO:00002005],189
1,air,env_broad_scale,cropland biome [ENVO:01000245],48
2,air,env_broad_scale,forest [ENVO:00000111],2
3,air,env_broad_scale,temperate grassland biome [ENVO:01000193],6
4,air,env_broad_scale,temperate woodland biome [ENVO:01000221],9
...,...,...,...,...
1585,water,ploidy,diploid [PATO:0001394],9
1586,water,ploidy,haploid [PATO:0001375],6
1587,water,ploidy,unmapped [unmapped],1
1588,water,samp_collect_device,bottle [FOODON:03490214],27


In [77]:
is_sums = (
    is_counts[["env_package", "variable", "val_count"]]
    .groupby(["env_package", "variable"])
    .agg(["sum"])
)
is_sums.reset_index(inplace=True)
is_sums.columns = ["env_package", "variable", "ep_var_sum"]

,env_package,variable,ep_var_sum
0,air,env_broad_scale,437
1,air,env_local_scale,437
2,air,env_medium,438
3,air,ploidy,2
4,built,env_broad_scale,14510
...,...,...,...
90,water,env_medium,18939
91,water,experimental_factor,410
92,water,host_body_product,103
93,water,ploidy,16


In [84]:
is_frac = pd.merge(is_counts, is_sums)
is_frac["fraction"] = is_frac["val_count"] / is_frac["ep_var_sum"]
is_frac = is_frac.drop("ep_var_sum", axis=1)
is_frac = is_frac.rename(columns={"variable": "slot"})

is_frac

,env_package,slot,value,val_count,fraction
0,air,env_broad_scale,air [ENVO:00002005],189,0.432494
1,air,env_broad_scale,cropland biome [ENVO:01000245],48,0.109840
2,air,env_broad_scale,forest [ENVO:00000111],2,0.004577
3,air,env_broad_scale,temperate grassland biome [ENVO:01000193],6,0.013730
4,air,env_broad_scale,temperate woodland biome [ENVO:01000221],9,0.020595
...,...,...,...,...,...
1585,water,ploidy,diploid [PATO:0001394],9,0.562500
1586,water,ploidy,haploid [PATO:0001375],6,0.375000
1587,water,ploidy,unmapped [unmapped],1,0.062500
1588,water,samp_collect_device,bottle [FOODON:03490214],27,0.030612


In [85]:
envpack_slot_fracs_file = "envpack_slot_fractions.tsv"
is_frac.to_csv(envpack_slot_fracs_file, index=False, sep="\t")

In [ ]:
casted.reset_index(level=0, inplace=True)

---

In [ ]:
# ba['ontology_prefix'].value_counts()

# ontology_elections = 'ENVO,MICRO,FMA,UBERON,NCBITAXON,FOODON,MA,EFO,CHEBI,AGRO,PATO,DOID,MONDO,OBI,PR,HP,PO'.lower()

 accounting of unfiltered search over top ~ 1000 query results

```
ENVO            191
NCIT            137
MICRO            34
OMIT             29
FMA              16
GAZ              11
UBERON           10
NCBITAXON         7
FOODON            6
MA                6
EFO               4
CHEBI             4
AGRO              4
SPD               3
CCO               3
GENEPIO           3
BAO               2
OHPI              2
OHMI              2
PATO              2
NMR               2
MS                2
DOID              2
IDOMAL            2
OGG               2
OBI               2
EOL               2
VTO               2
PR                1
MRO               1
MCO               1
HP                1
ARO               1
CO_357            1
ECAO              1
IDO-COVID-19      1
EDAM              1
MAXO              1
CO_366            1
BTO               1
TGMA              1
EUPATH            1
ExO               1
CEPH              1
PLANP             1
MIRO              1
FIDEO             1
COVOC             1
SYMP              1
HCAO              1
WBPhenotype       1
SO                1
ONS               1
EMAPA             1
AFO               1
ECTO              1
CO_340            1
```

----

we can now say what is meant by the presence of term ids in many of the annotations

Questions:
- should we compare the asserted label to the depleted annotation?
- over what space should we search the values for which a term/label wasn't found?
- use rdftab, runner or API like OLS?
- how to rank and filter hits?
- try harder with some of the malformed extracts?
- what about PREFIX:label style? remove prefixes that have already been recognized from the extracts?


----

Slow and unsuccessful attempt to cluster singletons

Using right-truncation of numbers instead

In [ ]:
# # slow

# lev_similarity = -1 * np.array(
#     [[distance.levenshtein(w1, w2) for w1 in words] for w2 in words]
# )

# lev_similarity

In [ ]:
# affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
# affprop.fit(lev_similarity)

```
/Users/MAM/Documents/gitrepos/scoped-mapping/venv/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(
/Users/MAM/Documents/gitrepos/scoped-mapping/venv/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:246: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "
```

In [ ]:
# for cluster_id in np.unique(affprop.labels_):
#     exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
#     cluster = np.unique(words[np.nonzero(affprop.labels_ == cluster_id)])
#     cluster_str = ", ".join(cluster)
#     print(" - *%s:* %s" % (exemplar, cluster_str))